In [1]:
import pyBigWig
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import gzip
import shutil
import os
import subprocess
import matplotlib.image as mpimg
import math 

In [2]:
peaks_folder = "/home/z5628505/Data/JoycData/NarrowPeaks"
output_directory = "/home/z5628505/Data/JoycData/Results"  # Define the directory here
bigwig_folder = "/home/z5628505/Data/JoycData/BigWig"

Creating an expanded 

In [6]:

# Ensure output directory is an absolute path
output_directory = os.path.abspath(output_directory)
os.makedirs(output_directory, exist_ok=True)  # Create directory once, not in the loop

reference_peak_files = [f for f in os.listdir(peaks_folder) if f.endswith(".narrowPeak")]

for rfile in reference_peak_files:
    # Load reference peak file
    reference_name = "_".join(rfile.split("_")[:2])

    # Construct the output file path
    output_file = os.path.join(output_directory, f"{reference_name}_expanded.bed")

    print(f"Saving expanded peaks to: {output_file}")

    # Load BED file (assuming no header, 10 columns for .narrowPeak)
    file_path = os.path.join(peaks_folder, rfile)
    df = pd.read_csv(file_path, sep="\t", header=None)

    # Ensure file has enough columns
    if df.shape[1] < 10:
        print(f"Skipping {rfile}: insufficient columns")
        continue

    # Compute summit position (column 9 contains summit offset)
    summit = df[1] + df[9]

    # Expand regions symmetrically
    df[1] = (summit - 500).clip(lower=0)  # Ensure no negative start positions
    df[2] = summit + 750

    # Save the expanded BED file
    df.to_csv(output_file, sep="\t", header=False, index=False)

    print(f"Expanded peaks saved to {output_file}")


Saving expanded peaks to: /home/z5628505/Data/JoycData/Results/LMO2_LMPP_expanded.bed
Expanded peaks saved to /home/z5628505/Data/JoycData/Results/LMO2_LMPP_expanded.bed
Saving expanded peaks to: /home/z5628505/Data/JoycData/Results/PU1_LMPP_expanded.bed
Expanded peaks saved to /home/z5628505/Data/JoycData/Results/PU1_LMPP_expanded.bed
Saving expanded peaks to: /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed
Expanded peaks saved to /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed
Saving expanded peaks to: /home/z5628505/Data/JoycData/Results/GATA2_LMPP_expanded.bed
Expanded peaks saved to /home/z5628505/Data/JoycData/Results/GATA2_LMPP_expanded.bed
Saving expanded peaks to: /home/z5628505/Data/JoycData/Results/LYL1_LMPP_expanded.bed
Expanded peaks saved to /home/z5628505/Data/JoycData/Results/LYL1_LMPP_expanded.bed
Saving expanded peaks to: /home/z5628505/Data/JoycData/Results/ERG_LMPP_expanded.bed
Expanded peaks saved to /home/z5628505/Data/JoycData/Results/ERG_

Make katrix with bigwig files and defined region

In [ ]:
# Get list of BigWig files
bw_files = [os.path.join(bigwig_folder, f) for f in os.listdir(bigwig_folder) if f.endswith(".bw")]

print("Found BigWig files:", bw_files)

# Get list of expanded BED files
expanded_bed_files = [os.path.join(output_directory, f) for f in os.listdir(output_directory) if f.endswith("_expanded.bed")]

print("Found expanded BED files:", expanded_bed_files)

for rfile in expanded_bed_files:  # Iterate over BED files
    reference_name = "_".join(os.path.basename(rfile).split("_")[:2])  # Extract reference name

    for bw_file in bw_files:
        bw_name = os.path.basename(bw_file).replace(".bw", "")
        sample_name = "_".join(bw_name.split("_")[:2])  # Extract first two prefixes
        
        matrix_file = os.path.join(output_directory, f"{reference_name}_{sample_name}_Matrix.gz")

        # Run computeMatrix
        command = [
            "computeMatrix", "reference-point",
            "-R", rfile, "-S", bw_file,
            "--skipZeros", "--outFileName", matrix_file
        ]

        print(f"Running: {' '.join(command)}")  # Print command for debugging
        subprocess.run(command, check=True)

Found BigWig files: ['/home/z5628505/Data/JoycData/BigWig/ERG_LMPP_JV74.bw', '/home/z5628505/Data/JoycData/BigWig/GATA2_LMPP_JV73.bw', '/home/z5628505/Data/JoycData/BigWig/TAL1_LMPP_JV68.bw', '/home/z5628505/Data/JoycData/BigWig/PU1_LMPP_JV75.bw', '/home/z5628505/Data/JoycData/BigWig/FLI1_LMPP_JV69.bw', '/home/z5628505/Data/JoycData/BigWig/LMO2_LMPP_JV71.bw', '/home/z5628505/Data/JoycData/BigWig/CTCF_LMPP_JV77.bw', '/home/z5628505/Data/JoycData/BigWig/RUNX1_LMPP_JV70.bw', '/home/z5628505/Data/JoycData/BigWig/LYL1_LMPP_JV72.bw', '/home/z5628505/Data/JoycData/BigWig/STAG2_LMPP_JV76.bw']
Found expanded BED files: ['/home/z5628505/Data/JoycData/Results/GATA2_LMPP_expanded.bed', '/home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed', '/home/z5628505/Data/JoycData/Results/LYL1_LMPP_expanded.bed', '/home/z5628505/Data/JoycData/Results/ERG_LMPP_expanded.bed', '/home/z5628505/Data/JoycData/Results/STAG2_LMPP_expanded.bed', '/home/z5628505/Data/JoycData/Results/TAL1_LMPP_expanded.bed', '/

Skipping CTCF_LMPP_JV77_peak_12511, due to being absent in the computeMatrix output.


Running: computeMatrix reference-point -R /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed -S /home/z5628505/Data/JoycData/BigWig/TAL1_LMPP_JV68.bw --skipZeros --outFileName /home/z5628505/Data/JoycData/Results/CTCF_LMPP_TAL1_LMPP_Matrix.gz


Skipping CTCF_LMPP_JV77_peak_9886, due to being absent in the computeMatrix output.
Skipping CTCF_LMPP_JV77_peak_18793b, due to being absent in the computeMatrix output.
Skipping CTCF_LMPP_JV77_peak_18793c, due to being absent in the computeMatrix output.


Running: computeMatrix reference-point -R /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed -S /home/z5628505/Data/JoycData/BigWig/PU1_LMPP_JV75.bw --skipZeros --outFileName /home/z5628505/Data/JoycData/Results/CTCF_LMPP_PU1_LMPP_Matrix.gz
Running: computeMatrix reference-point -R /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed -S /home/z5628505/Data/JoycData/BigWig/FLI1_LMPP_JV69.bw --skipZeros --outFileName /home/z5628505/Data/JoycData/Results/CTCF_LMPP_FLI1_LMPP_Matrix.gz
Running: computeMatrix reference-point -R /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed -S /home/z5628505/Data/JoycData/BigWig/LMO2_LMPP_JV71.bw --skipZeros --outFileName /home/z5628505/Data/JoycData/Results/CTCF_LMPP_LMO2_LMPP_Matrix.gz
Running: computeMatrix reference-point -R /home/z5628505/Data/JoycData/Results/CTCF_LMPP_expanded.bed -S /home/z5628505/Data/JoycData/BigWig/CTCF_LMPP_JV77.bw --skipZeros --outFileName /home/z5628505/Data/JoycData/Results/CTCF_LMPP_CTCF_LMPP_Matr

In [ ]:

# Define output directory where matrix files are stored
matrix_directory = output_directory  # This is where matrices are stored
heatmap_directory = output_directory  # This is where heatmaps will be saved

# Get all .gz files in the output directory
matrix_files = [os.path.join(matrix_directory, f) for f in os.listdir(matrix_directory) if f.endswith(".gz")]

print("Found matrix files:", matrix_files)

for matrix_file in matrix_files:
    print(f"Processing {matrix_file}")

    # Read matrix file directly from .gz
    with gzip.open(matrix_file, "rt", encoding="utf-8", errors="replace") as f:
        df = pd.read_csv(f, sep="\t", comment='@', header=None)

    # Check the first few rows to inspect the structure
    print(f"First few rows of {matrix_file}:")
    print(df.head())

    # Create the heatmap
    matrix_name = os.path.basename(matrix_file).replace("_matrix.gz", "")
       # Define both PNG and SVG output paths
    heatmap_output_png = os.path.join(heatmap_directory, f"{matrix_name}_heatmap.png")
    heatmap_output_svg = os.path.join(heatmap_directory, f"{matrix_name}_heatmap.svg")

    # Command for PNG output
   # command_png = [
       # "plotHeatmap",
       # "-m", matrix_file,  # Path to the matrix file
       # "-out", heatmap_output_png,  # Output path for PNG
        #"--colorMap", "RdYlBu",  # Color map
    #]

    # Command for SVG output
    command_svg = [
        "plotHeatmap",
        "-m", matrix_file,  # Path to the matrix file
        "-out", heatmap_output_svg,  # Output path for SVG
        "--colorMap", "RdYlBu",  # Color map
    ]

    # Run both commands
    for command in [command_svg]:
        print(f"Running command: {' '.join(command)}")  # Debugging info
        result = subprocess.run(command, check=False, capture_output=True, text=True)
        print(result.stderr)  # Print any errors

    